In [1]:
import uavgeo as ug
import geopandas as gpd
import rioxarray
import numpy as np
import ultralytics
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
import cv2
import os
import pandas as pd
from PIL import Image

c:\users\judoj\documents\programming\uavgeo\uavgeo\compute\products.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# pix2pix result analysis

1. Data loading and resetting to geographic coordinates using the test-splits
2. Visualizing the correlation of epoch200 vs real (with density)
3. Visualizing the epoch improvements
4. Visualizing rgb-true-p2p_e200 

In [2]:
# I just adjusted these everytim for all the runs

In [48]:
# load the datasplits 
cwd = os.getcwd()

chips = gpd.read_file("../data/canyelles_chips_256.geojson")

# load the original orthomosaic 
ortho = rxr.open_rasterio("../data/canyelles/orthos/rgb_230609.tif",mode = "w")

#alignment (if required)
r= rxr.open_rasterio("../data/canyelles/orthos/cropped_r_230609.tif",mode = "w")
ortho = ortho.rio.reproject_match(r).sel(band=[1,2,3])
single_band  = r

#split the ndvi raster to acquire the boundaries, xylims etc.
test_rgbs = [single_band.rio.clip_box(minx = row.geometry.bounds[0], miny =row.geometry.bounds[1] ,maxx =row.geometry.bounds[2] , maxy= row.geometry.bounds[3]) for i, row in chips.iterrows()]


In [49]:
import re

def check_pattern_matches(pattern, string_list):
    matched_strings = []
    for string in string_list:
        if re.search(pattern, string):
            matched_strings.append(string)
    return matched_strings


## 1. Reconstruct the P2P imagery

In [50]:
#Reconstruct an epoch that has passed thorugh p2p
folder = "../data/generated_ndvi/canyelles_trgb_ndvi_256"

fakeb = "fake_B"
realb = "real_B"
reala = "real_A"

        
imgs = [os.path.join(img) for img in os.listdir(folder)]
real_a =[os.path.join(folder, file) for file in check_pattern_matches(reala, imgs)]
real_b = [os.path.join(folder, file) for file in check_pattern_matches(realb, imgs)]
fake_b = [os.path.join(folder, file) for file in check_pattern_matches(fakeb, imgs)]
         

In [53]:
# load them as images

import numpy as np
from PIL import Image

def image_to_numpy_array(image_path):
    # Open the image using Pillow
    image = Image.open(image_path)

    # Convert the image to a NumPy array
    numpy_array = np.array(image)

    return numpy_array
real_a_arr =[image_to_numpy_array(file) for file in real_a]
real_b_arr = [image_to_numpy_array(file) for file in real_b]
fake_b_arr = [image_to_numpy_array(file) for file in fake_b]
         

In [54]:
# fix dimension error due to jpg and png loading of the 512/p2pHD model
#fake_b_arr = [arr.reshape((512,512,1)) for arr in fake_b_arr]
#real_b_arr = [arr.reshape((512,512,1)) for arr in real_b_arr]

In [55]:
# empty rgb array from ortho:

empty_rgb = xr.full_like(ortho.sel(band=[1,2,3]).astype(np.uint8), fill_value=np.nan)
empty_ndvi = xr.full_like(single_band, fill_value=np.nan)


C:\Users\judoj\mambaforge\envs\ndvi\Lib\site-packages\numpy\core\numeric.py:407: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')


In [56]:
def np_chip_to_rxr(darray, geom, crs):

    min_x, min_y, max_x, max_y = geom.geometry.bounds
    
    x_coords = np.linspace(min_x, max_x, darray.shape[1])
    y_coords =np.linspace(min_y, max_y, darray.shape[0])[::-1]
    #darray = np.rot90(darray)
    return xr.DataArray(darray, dims=("y", "x", "band"), coords={"x": x_coords, "y": y_coords}).rio.write_crs(crs).transpose('band', 'y', 'x')


In [57]:
from rioxarray import merge

rgb_test_list = [np_chip_to_rxr(real_a_arr[i], row, chips.crs) for i, row in chips.iterrows()]
#add the empty_rgb to the list
rgb_test = merge.merge_arrays([empty_rgb] + rgb_test_list)

# FOR NDVI
ndvi_test_list = [np_chip_to_rxr(real_b_arr[i], row, chips.crs).sel(band=0) for i, row in chips.iterrows()]
#add the empty_ndvi to the list
ndvi_test = merge.merge_arrays([empty_ndvi] + ndvi_test_list)   

# FOR P2P NDVI
ndvi_p2p_list = [np_chip_to_rxr(fake_b_arr[i], row, chips.crs).sel(band=0) for i, row in chips.iterrows()]
#add the empty_rgb to the list
ndvi_p2p = merge.merge_arrays([empty_ndvi] + ndvi_p2p_list)   

In [58]:
ndvi_p2p.rio.crs

CRS.from_epsg(4326)

In [59]:
# Clip to test-area to reduce array size
test=chips

ndvi_p2p = ndvi_p2p.rio.clip_box(minx = test.bounds.min().minx, miny =test.bounds.min().miny ,maxx =test.bounds.max().maxx , maxy= test.bounds.max().maxy) 

ndvi_test = ndvi_test.rio.clip_box(minx = test.bounds.min().minx, miny =test.bounds.min().miny ,maxx =test.bounds.max().maxx , maxy= test.bounds.max().maxy) 

rgb_test = rgb_test.rio.clip_box(minx = test.bounds.min().minx, miny =test.bounds.min().miny ,maxx =test.bounds.max().maxx , maxy= test.bounds.max().maxy) 

# Scale ndvi to -1 : 1 
ndvi_p2p = ndvi_p2p.where(ndvi_p2p>0)
ndvi_p2p = (ndvi_p2p/127.5)-1

ndvi_test = ndvi_test.where(ndvi_test>0)
ndvi_test = (ndvi_test/127.5)-1


In [ ]:

# export for some visual inspection in qgis

rgb_test.rio.to_raster("../data/orthos/reconstructed/canyelles_trgb.tif")
ndvi_test.rio.to_raster("../data/orthos/reconstructed/canyelles_ndvi_real.tif")
ndvi_p2p.rio.to_raster("../data/orthos/reconstructed/canyelles_ndvi_p2p.tif")